-----------------------------------------------------------------------------------------

IMPORTS

-------------------------------------------------------------------------------

In [1]:
# imports
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from openpyxl import load_workbook
from openpyxl.styles import NamedStyle
from openpyxl.utils import get_column_letter
from urllib.parse import quote
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import requests

-----------------------------------------------------------------

PROXY CHECK

-----------------------------------------------------------------------


In [2]:
with open('proxy_list.txt', 'r') as file:
    proxy_list = file.readlines()  # Liest die Liste der Proxies aus der Datei  (Die txt Datei mit den Proxys ist eine Liste von Free Proxys aus dem Internet)

working_proxies = []  # Eine leere Liste für funktionierende Proxies wird erstellt

for proxy in proxy_list:  # Iteration über jeden Proxy in der Liste
    proxy = proxy.strip()  # Entfernt Leerzeichen und Zeilenumbrüche am Anfang und Ende des Proxies
    try:
        response = requests.get('https://www.google.com', proxies={'https': proxy}, timeout=5)
        # Sendet eine Anfrage an Google mit dem Proxy und einem Timeout von 5 Sekunden

        if response.status_code == 200:  # Überprüft den Statuscode der Antwort
            working_proxies.append(proxy)  # Fügt den Proxy zur Liste der funktionierenden Proxies hinzu
    except requests.exceptions.RequestException:
        pass
        # Behandelt Ausnahmen, die während der Anfrage auftreten können, und fährt fort

-----------------------------------------------------------------------

GOOGLE WEBSCRAPING

-----------------------------------------------------------------------

In [ ]:
aktueller_index = 0  # Variable zur Verfolgung des aktuellen Index im DataFrame
excel_datei = 'Excel mit suchbegriffen in der ersten Spalte.xlsx'  # Pfad zur Excel-Datei
df = pd.read_excel(excel_datei)  # Einlesen der Excel-Datei in einen DataFrame
spalte_hersteller_ccp = 'Hersteller_CCP'  # Name der Spalte, die den Hersteller enthält

# Erstellung der Proxy-Optionen für den Chrome Webdriver
proxy_options = webdriver.ChromeOptions()

# Initialisierung des Chrome Webdrivers mit den Proxy-Optionen
driver = webdriver.Chrome(options=proxy_options)
driver.get("https://www.google.com")  # Öffnen der Google-Startseite im Webbrowser

# Klick auf den Zustimmungs-Button für Google-Cookies (sofern vorhanden)
try:
    button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.ID, "L2AGLb"))) # Cookie Akzeptier Bottom finden (als Variable speichern )
    button.click() # Akzeptier buttom klicken 
except:
    pass

neue_excel_datei = 'Gewünschter Name.xlsx'  # Name der neuen Excel-Datei
wb = load_workbook(excel_datei)  # Laden der vorhandenen Excel-Datei
wb.save(neue_excel_datei)  # Speichern der Excel-Datei unter neuem Namen

neuer_df = pd.read_excel(neue_excel_datei)  # Einlesen der neuen Excel-Datei in einen DataFrame

while aktueller_index < len(df):  # Schleife für jeden Eintrag im DataFrame
    wert = df.at[aktueller_index, spalte_hersteller_ccp]  # Wert in der aktuellen Zeile und Spalte abrufen
    if pd.isnull(wert):  # Überprüfen, ob der Wert leer ist
        break  # Schleife beenden, wenn der Wert leer ist

    random_proxy = random.choice(working_proxies)  # Auswahl eines zufälligen funktionierenden Proxys
    proxy_options.add_argument(f'--proxy-server={random_proxy}')  # Hinzufügen des Proxy-Servers zu den Optionen des Chrome Webdrivers

    url = f"https://www.google.com/search?q={wert}"  # URL für die Google-Suche mit dem aktuellen Wert

    try:
        driver.get(url)  # Öffnen der Google-Suche im Webbrowser
        wait = WebDriverWait(driver, 5)  # Warten auf das Laden der Seite (möglicherweise zu kurz)
        wait.until(EC.presence_of_element_located((By.ID, 'kp-wp-tab-overview')))  # Warten auf das Laden des Suchergebnisses
        I1 = driver.find_elements(By.CLASS_NAME, 'rVusze')  # Finden der Suchergebnis-Elemente
        for x in I1:
            ausgabe = x.text # text als Var speichern
            trenn_index = ausgabe.find(':') # Trennstellen als Var speichern
            spaltenname = ausgabe[:trenn_index] # Spaltelnamen festlegen
            zelleninhalt = ausgabe[trenn_index + 1:] # Zelleninhalte festlegen
            neuer_df.at[aktueller_index, spaltenname] = zelleninhalt  # Aktualisieren des Werts in der neuen Excel-Datei
    except:
        pass

    aktueller_index += 1  # Inkrementieren des Index für die nächste Zeile

neuer_df.to_excel(neue_excel_datei, index=False)  # Speichern des aktualisierten DataFrames in der neuen Excel-Datei
driver.quit()  # Beenden des Chrome Webdrivers
